In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tfrecorder import TFrecorder
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%pylab inline

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Populating the interactive namespace from numpy and matplotlib


In [2]:
tfr = TFrecorder()
def input_fn_maker(path, data_info_path, shuffle=False, batch_size = 1, epoch = 1, padding = None):
    def input_fn():
        filenames = tfr.get_filenames(path=path, shuffle=shuffle)
        dataset=tfr.get_dataset(paths=filenames, data_info=data_info_path, shuffle = shuffle, 
                            batch_size = batch_size, epoch = epoch, padding =padding)
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()
    return input_fn

padding_info = ({'image':[784,],'label':[]})
test_input_fn = input_fn_maker('mnist_tfrecord/test/',  'mnist_tfrecord/data_info.csv',batch_size = 512,
                               padding = padding_info)
train_input_fn = input_fn_maker('mnist_tfrecord/train/',  'mnist_tfrecord/data_info.csv', shuffle=True, batch_size = 128,
                               padding = padding_info)
train_eval_fn = input_fn_maker('mnist_tfrecord/train/',  'mnist_tfrecord/data_info.csv', batch_size = 512,
                               padding = padding_info)

In [3]:
def model_fn(features, mode):
    # reshape 784维的图片到28x28的平面表达，1为channel数
    features['image'] = tf.reshape(features['image'],[-1,28,28,1])
    # shape: [None,28,28,1]
    conv1 = tf.layers.conv1d(
            inputs = tf.reshape(features['image'],[-1,28,28]), 
            filters = 32, 
            kernel_size = 5,
            padding="same",
            activation=tf.nn.relu,
            name = 'conv1')
    # shape: [None,28,32]
    pool1 = tf.layers.max_pooling1d(inputs = conv1, 
                          pool_size=2,
                          strides=2,
                          name = 'pool1')
    # shape: [None,14,32]
    # create RNN cells:
    rnn_cells = [tf.nn.rnn_cell.GRUCell(dim,kernel_initializer=tf.orthogonal_initializer) for dim in [128,256]]
    # stack cells for multi-layers RNN
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_cells)
    # create RNN layers
    outputs, last_state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                   inputs=pool1,
                                   dtype=tf.float32)
    # shape: outputs: [None,14,256]
    # shape: last_state: [None,256]
    dense1 = tf.layers.dense(inputs=last_state[1], units=1024, activation=tf.nn.relu, name= 'dense1')
    # shape: [None,1024]
    dropout = tf.layers.dropout(inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dense1, units=10, name= 'output')
    # shape: [None,10]
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "labels": features['label'],
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
        }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=features['label'], logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=features['label'], predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
rm -rf mnist_model_cnn_rnn/

In [5]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir="mnist_model_cnn_rnn")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'mnist_model_cnn_rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f29db388908>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
mnist_classifier.train(
    input_fn=train_input_fn)

read dataframe from mnist_tfrecord/train/train42000_43000.tfrecord x 55
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into mnist_model_cnn_rnn/model.ckpt.
INFO:tensorflow:loss = 2.29675, step = 1
INFO:tensorflow:global_step/sec: 14.6744
INFO:tensorflow:loss = 0.43611, step = 101 (6.815 sec)
INFO:tensorflow:global_step/sec: 15.0812
INFO:tensorflow:loss = 0.112588, step = 201 (6.631 sec)
INFO:tensorflow:global_step/sec: 15.2808
INFO:tensorflow:loss = 0.118365, step = 301 (6.545 sec)
INFO:tensorflow:global_step/sec: 15.0366
INFO:tensorflow:loss = 0.311664, step = 401 (6.651 sec)
INFO:tensorflow:Saving checkpoints for 430 into mnist_model_cnn_rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0546862.


In [7]:
eval_results = mnist_classifier.evaluate(input_fn=train_eval_fn)
print('train set')
print(eval_results)

read dataframe from mnist_tfrecord/train/train19000_20000.tfrecord x 55
INFO:tensorflow:Starting evaluation at 2018-04-07-14:09:08
INFO:tensorflow:Restoring parameters from mnist_model_cnn_rnn/model.ckpt-430
INFO:tensorflow:Finished evaluation at 2018-04-07-14:09:15
INFO:tensorflow:Saving dict for global step 430: accuracy = 0.955836, global_step = 430, loss = 0.142296
train set
{'accuracy': 0.95583636, 'loss': 0.14229576, 'global_step': 430}


In [8]:
eval_results = mnist_classifier.evaluate(input_fn=test_input_fn)
print('test set')
print(eval_results)

read dataframe from mnist_tfrecord/test/test1000_2000.tfrecord x 10
INFO:tensorflow:Starting evaluation at 2018-04-07-14:09:16
INFO:tensorflow:Restoring parameters from mnist_model_cnn_rnn/model.ckpt-430
INFO:tensorflow:Finished evaluation at 2018-04-07-14:09:18
INFO:tensorflow:Saving dict for global step 430: accuracy = 0.9576, global_step = 430, loss = 0.130185
test set
{'accuracy': 0.9576, 'loss': 0.1301849, 'global_step': 430}


In [9]:
# train 5 more epochs
for i in range(5):
    mnist_classifier.train(
        input_fn=train_input_fn)

read dataframe from mnist_tfrecord/train/train9000_10000.tfrecord x 55
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from mnist_model_cnn_rnn/model.ckpt-430
INFO:tensorflow:Saving checkpoints for 431 into mnist_model_cnn_rnn/model.ckpt.
INFO:tensorflow:loss = 0.0895679, step = 431
INFO:tensorflow:global_step/sec: 14.29
INFO:tensorflow:loss = 0.0751697, step = 531 (6.999 sec)
INFO:tensorflow:global_step/sec: 15.0418
INFO:tensorflow:loss = 0.118345, step = 631 (6.648 sec)
INFO:tensorflow:global_step/sec: 15.1348
INFO:tensorflow:loss = 0.0903634, step = 731 (6.607 sec)
INFO:tensorflow:global_step/sec: 15.1443
INFO:tensorflow:loss = 0.0965481, step = 831 (6.603 sec)
INFO:tensorflow:Saving checkpoints for 860 into mnist_model_cnn_rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0702864.
read dataframe from mnist_tfrecord/train/train23000_24000.tfrecord x 55
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from mnist_mo

In [10]:
eval_results = mnist_classifier.evaluate(input_fn=train_eval_fn)
print('train set')
print(eval_results)

read dataframe from mnist_tfrecord/train/train19000_20000.tfrecord x 55
INFO:tensorflow:Starting evaluation at 2018-04-07-14:13:34
INFO:tensorflow:Restoring parameters from mnist_model_cnn_rnn/model.ckpt-2580
INFO:tensorflow:Finished evaluation at 2018-04-07-14:13:40
INFO:tensorflow:Saving dict for global step 2580: accuracy = 0.985855, global_step = 2580, loss = 0.0434273
train set
{'accuracy': 0.98585457, 'loss': 0.043427251, 'global_step': 2580}


In [11]:
eval_results = mnist_classifier.evaluate(input_fn=test_input_fn)
print('test set')
print(eval_results)

read dataframe from mnist_tfrecord/test/test1000_2000.tfrecord x 10
INFO:tensorflow:Starting evaluation at 2018-04-07-14:13:43
INFO:tensorflow:Restoring parameters from mnist_model_cnn_rnn/model.ckpt-2580
INFO:tensorflow:Finished evaluation at 2018-04-07-14:13:44
INFO:tensorflow:Saving dict for global step 2580: accuracy = 0.9819, global_step = 2580, loss = 0.063138
test set
{'accuracy': 0.98189998, 'loss': 0.063137986, 'global_step': 2580}
